# Deploying functions 

In [90]:
#Reference
#https://docs.openfaas.com/tutorials/kubernetes-hpa/
#!ssh ubuntu@172.17.141.197 "sudo kubectl port-forward -n openfaas svc/gateway 8080:8080 &"
import warnings

warnings.filterwarnings('ignore')

In [91]:
#Deploy function and enable the function auto scaling by openfaas
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f aeneas.yml  --env max_inflight=50 --annotation topic="aeneas" --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1"

Deploying: aeneas.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/aeneas



In [92]:
#Deploy function and enable the function auto scaling by openfaas
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f scaling-aeneas-tocloud.yml --env max_inflight=50 --annotation topic="tocloud" --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1;faas deploy  -f getobject.yml --env max_inflight=50 --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1;faas deploy  -f pubrabbitmq.yml --env max_inflight=50 --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1"


Deploying: scaling-aeneas-tocloud.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/scaling-aeneas-tocloud

Deploying: getobject.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/getobject

Deploying: pubrabbitmq.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/pubrabbitmq



In [93]:
!ssh ubuntu@172.17.141.197 "faas describe getobject"

Name:               getobject
Status:             Ready
Replicas:           1
Available Replicas: 1
Invocations:        14017
Image:              shivupoojar/scalinggetobject:latest
Function Process:   python index.py
URL:                http://127.0.0.1:8080/function/getobject
Async URL:          http://127.0.0.1:8080/async-function/getobject
Labels:
 com.openfaas.scale.min: 1
 faas_function: getobject
 uid: 391782603
 com.openfaas.scale.max: 1
Annotations:
 prometheus.io.scrape: false
Requests:  CPU: 500m
 Memory: 128Mi


# Deploying Rabbitmq Connector 

In [94]:
# Deploying connector configurartion
!ssh ubuntu@172.17.141.197 "cd rabbitmq-connector/artifacts;sudo kubectl apply -f cf.yaml"

configmap/rabbitmq-connector-configmap unchanged


In [95]:
# Deploy connector
!ssh ubuntu@172.17.141.197 "cd rabbitmq-connector/artifacts;sudo kubectl apply -f configmap.yaml"

deployment.apps/rabbitmq-connector configured


In [96]:
!ssh ubuntu@172.17.141.197 "sudo kubectl get po -n openfaas" 

NAME                                  READY   STATUS              RESTARTS       AGE
nats-5b77986b8-48p2x                  1/1     Running             4 (6d5h ago)   42d
queue-worker-7d8668444f-tmq7b         1/1     Running             5 (6d5h ago)   28d
alertmanager-6cd7bd5c64-hrcrt         1/1     Running             4 (6d5h ago)   28d
basic-auth-plugin-78b958969c-tnpck    1/1     Running             4 (6d5h ago)   28d
gateway-bf4f5d6f7-4q7kn               2/2     Running             10 (5d ago)    28d
prometheus-7678c644d6-k28wr           1/1     Running             0              8h
rabbitmq-connector-64cbd548f5-w5bgr   0/1     ContainerCreating   0              2s


# Deploying k8s HPA components 

In [97]:
!ssh ubuntu@172.17.141.197 "sudo kubectl autoscale deployment -n openfaas-fn aeneas --cpu-percent=30 --min=1 --max=15;sudo kubectl autoscale deployment -n openfaas-fn scaling-aeneas-tocloud --cpu-percent=50 --min=1 --max=10;sudo kubectl autoscale deployment -n openfaas-fn getobject --cpu-percent=50 --min=1 --max=10;sudo kubectl autoscale deployment -n openfaas-fn pubrabbitmq --cpu-percent=50 --min=1 --max=10"

horizontalpodautoscaler.autoscaling/aeneas autoscaled
horizontalpodautoscaler.autoscaling/scaling-aeneas-tocloud autoscaled
horizontalpodautoscaler.autoscaling/getobject autoscaled
horizontalpodautoscaler.autoscaling/pubrabbitmq autoscaled


In [98]:
#;sudo kubectl autoscale deployment -n openfaas rabbitmq-connector --cpu-percent=50 --min=1 --max=10;

In [99]:
#Check for the created HPA
!ssh ubuntu@172.17.141.197 "sudo kubectl get hpa -n openfaas-fn"

NAME                     REFERENCE                           TARGETS         MINPODS   MAXPODS   REPLICAS   AGE
aeneas                   Deployment/aeneas                   <unknown>/30%   1         15        0          3s
scaling-aeneas-tocloud   Deployment/scaling-aeneas-tocloud   <unknown>/50%   1         10        0          3s
getobject                Deployment/getobject                <unknown>/50%   1         10        0          3s
pubrabbitmq              Deployment/pubrabbitmq              <unknown>/50%   1         10        0          2s


# Experiments 

In [100]:
import sys
import  datetime
import os
from minio import Minio
import csv
import json
import pandas as pd
import time
import subprocess
from subprocess import Popen, PIPE
import requests
from IPython.display import display

In [101]:
scenario = "k8s_keda"
user=10
iteration=0.05
minio_host="172.17.141.197:9001"
bucket_name="aeneas-output"
PROMETHEUS = 'http://172.17.141.197:31022/'
time_locust=120


In [102]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    client.remove_object(bucket_name, obj.object_name)

In [103]:
!rm input_data.csv
!touch input_data.csv
!echo "intime" > input_data.csv

In [104]:
!locust --headless -f testingrabbitlocust.py --host='127.0.0.1' --skip-log -t 120s  --users 10 --spawn-rate 0.05

In [105]:
time.sleep(60)

In [106]:
df= pd.read_csv('input_data.csv')
print(df)

                          intime
0   '2023-01-17 19:24:37.396936'
1   '2023-01-17 19:24:52.039883'
2   '2023-01-17 19:24:57.394757'
3   '2023-01-17 19:24:59.437512'
4   '2023-01-17 19:25:05.252118'
5   '2023-01-17 19:25:07.297605'
6   '2023-01-17 19:25:13.355344'
7   '2023-01-17 19:25:17.387654'
8   '2023-01-17 19:25:19.109523'
9   '2023-01-17 19:25:23.443942'
10  '2023-01-17 19:25:23.555842'
11  '2023-01-17 19:25:31.355381'
12  '2023-01-17 19:25:31.485097'
13  '2023-01-17 19:25:37.391657'
14  '2023-01-17 19:25:38.097678'
15  '2023-01-17 19:25:43.849082'
16  '2023-01-17 19:25:45.774190'
17  '2023-01-17 19:25:45.990830'
18  '2023-01-17 19:25:47.629130'
19  '2023-01-17 19:25:50.752429'
20  '2023-01-17 19:25:56.771392'
21  '2023-01-17 19:25:57.177763'
22  '2023-01-17 19:25:57.393556'
23  '2023-01-17 19:25:58.154799'
24  '2023-01-17 19:25:58.742499'
25  '2023-01-17 19:26:06.054776'
26  '2023-01-17 19:26:07.178625'
27  '2023-01-17 19:26:08.119264'
28  '2023-01-17 19:26:09.919090'
29  '2023-

In [107]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    outtime.append(obj.last_modified)#
    client.remove_object(bucket_name, obj.object_name)


In [108]:
outtime
df

,intime
0,'2023-01-17 19:24:37.396936'
1,'2023-01-17 19:24:52.039883'
2,'2023-01-17 19:24:57.394757'
3,'2023-01-17 19:24:59.437512'
4,'2023-01-17 19:25:05.252118'
5,'2023-01-17 19:25:07.297605'
6,'2023-01-17 19:25:13.355344'
7,'2023-01-17 19:25:17.387654'
8,'2023-01-17 19:25:19.109523'
9,'2023-01-17 19:25:23.443942'


In [109]:

df['outtime']= outtime
df['outtime']= pd.to_datetime(df['outtime'],errors='coerce').dt.tz_convert(None)
df['intime']= pd.to_datetime(df['intime'],errors='coerce')
#df =df.sort_values(by="outtime")
display(df)
print(df['outtime'].iloc[-1]-df['intime'].iloc[0])

,intime,outtime
0,2023-01-17 19:24:37.396936,2023-01-17 19:24:46.887
1,2023-01-17 19:24:52.039883,2023-01-17 19:24:56.274
2,2023-01-17 19:24:57.394757,2023-01-17 19:25:01.808
3,2023-01-17 19:24:59.437512,2023-01-17 19:25:05.420
4,2023-01-17 19:25:05.252118,2023-01-17 19:25:09.603
5,2023-01-17 19:25:07.297605,2023-01-17 19:25:13.251
6,2023-01-17 19:25:13.355344,2023-01-17 19:25:17.773
7,2023-01-17 19:25:17.387654,2023-01-17 19:25:22.869
8,2023-01-17 19:25:19.109523,2023-01-17 19:25:25.269
9,2023-01-17 19:25:23.443942,2023-01-17 19:25:27.810


0 days 00:02:28.666064


In [110]:
def get_function_execution_time(cmd):
    with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
        return (process.communicate()[0].decode("utf-8"))


In [111]:
#cmd1 = "sudo kubectl logs gateway-7b8d9dbb5b-rhnwl  -n openfaas -c gateway | grep getobject | cut -c 71-77 | tail -"
cmd1="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn  -n openfaas -c gateway  | grep /function/aeneas | grep 200 | cut -c 80-83 | tail -"+str(len(outtime))
cmd2="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/scaling-aeneas-tocloud | cut -c 96-101 | tail -"+str(len(outtime))
cmd3="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/getobject | grep POST | cut -c 71-76  | tail -"+str(len(outtime))
cmd4="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/pubrabbitmq | grep POST | cut -c 73-78 | tail -"+str(len(outtime))
df['aeneas']=(get_function_execution_time(cmd1)).split("\n")[:-1]
#df['aeneas']=(pd.to_timedelta(df['aeneas'].astype(float), unit='s'))

df['tocloud']=(get_function_execution_time(cmd2)).split("\n")[:-1]
#df['tocloud']=(pd.to_timedelta(df['tocloud'].astype(float), unit='s'))

#df['getobject']=(get_function_execution_time(cmd3)).split("\n")[:-1]
df['pubrabbitmq']=((get_function_execution_time(cmd4)).split("\n")[:-1])
#df['pubrabbitmq']=(pd.to_timedelta(df['pubrabbitmq'].astype(float), unit='s'))
df

,intime,outtime,aeneas,tocloud,pubrabbitmq
0,2023-01-17 19:24:37.396936,2023-01-17 19:24:46.887,4.28,0.0428,0.0408
1,2023-01-17 19:24:52.039883,2023-01-17 19:24:56.274,4.14,0.0412,0.0368
2,2023-01-17 19:24:57.394757,2023-01-17 19:25:01.808,4.31,0.0488,0.0382
3,2023-01-17 19:24:59.437512,2023-01-17 19:25:05.420,5.89,0.0443,0.0375
4,2023-01-17 19:25:05.252118,2023-01-17 19:25:09.603,4.25,0.0389,0.0453
5,2023-01-17 19:25:07.297605,2023-01-17 19:25:13.251,5.87,0.0391,0.0397
6,2023-01-17 19:25:13.355344,2023-01-17 19:25:17.773,4.32,0.0560,0.0347
7,2023-01-17 19:25:17.387654,2023-01-17 19:25:22.869,5.40,0.0318,0.0363
8,2023-01-17 19:25:19.109523,2023-01-17 19:25:25.269,6.08,0.0427,0.0283
9,2023-01-17 19:25:23.443942,2023-01-17 19:25:27.810,4.27,0.0378,0.0534


In [112]:
#df =df.sort_values(by="outtime")
df['TPT']=(df['outtime']-df['intime'])
df['TPT']=df['TPT'].values.astype('datetime64[ns]')
df['TPT']= pd.to_datetime(df['TPT']).dt.strftime('%S.%f')
df['TPT']=df['TPT'].astype(float)
df['PT']= df['TPT'].round(decimals = 0)
df['FET'] = (df['aeneas']).astype(float) + (df['tocloud']).astype(float)
#df['MQ time']= ((df['TPT']).astype(float) - df['FET'])
df["scenario"]= ""+str(user)+"_"+str(iteration)+"_"+scenario
display(df)
#TPT: Total Processing Time
#FET: Function Execution Time
#CT: Communication Time
#print(df)
# #df =df.sort_values(by="outtime")
# df['TPT']=(df['outtime']-df['intime'])
# #print(df['TPT'])
# #df['aeneas']= pd.to_timedelta(df['aeneas'].astype(float) , unit='s')
# df['FET'] = df['aeneas'].astype(float) + df['tocloud'].astype(float)
# #df['FET'] = pd.to_timedelta(df['aeneas'].astype(float), unit='s') + pd.to_timedelta(df['tocloud'].astype(float) , unit='s')
# df['FET']= pd.to_timedelta(df['FET'], unit='s')
# df['MQ time']= (df['TPT']) - (df['FET'])
# df["scenario"]= ""+str(user)+"_"+str(iteration)+"_"+scenario
# display(df)
df.to_csv(""+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv')
print(df.index)

,intime,outtime,aeneas,tocloud,pubrabbitmq,TPT,PT,FET,scenario
0,2023-01-17 19:24:37.396936,2023-01-17 19:24:46.887,4.28,0.0428,0.0408,9.490064,9.0,4.3228,10_0.05_k8s_keda
1,2023-01-17 19:24:52.039883,2023-01-17 19:24:56.274,4.14,0.0412,0.0368,4.234117,4.0,4.1812,10_0.05_k8s_keda
2,2023-01-17 19:24:57.394757,2023-01-17 19:25:01.808,4.31,0.0488,0.0382,4.413243,4.0,4.3588,10_0.05_k8s_keda
3,2023-01-17 19:24:59.437512,2023-01-17 19:25:05.420,5.89,0.0443,0.0375,5.982488,6.0,5.9343,10_0.05_k8s_keda
4,2023-01-17 19:25:05.252118,2023-01-17 19:25:09.603,4.25,0.0389,0.0453,4.350882,4.0,4.2889,10_0.05_k8s_keda
5,2023-01-17 19:25:07.297605,2023-01-17 19:25:13.251,5.87,0.0391,0.0397,5.953395,6.0,5.9091,10_0.05_k8s_keda
6,2023-01-17 19:25:13.355344,2023-01-17 19:25:17.773,4.32,0.0560,0.0347,4.417656,4.0,4.3760,10_0.05_k8s_keda
7,2023-01-17 19:25:17.387654,2023-01-17 19:25:22.869,5.40,0.0318,0.0363,5.481346,5.0,5.4318,10_0.05_k8s_keda
8,2023-01-17 19:25:19.109523,2023-01-17 19:25:25.269,6.08,0.0427,0.0283,6.159477,6.0,6.1227,10_0.05_k8s_keda
9,2023-01-17 19:25:23.443942,2023-01-17 19:25:27.810,4.27,0.0378,0.0534,4.366058,4.0,4.3078,10_0.05_k8s_keda


RangeIndex(start=0, stop=44, step=1)


In [113]:
pre_url = PROMETHEUS + '/api/v1/query_range?query='
timestamp= pd.to_datetime(df['outtime'].iloc[-1], format='%Y-%m-%d %H:%M:%S')
time_interval = (time.mktime(timestamp.timetuple()))
user=df.shape[0]
end=(time.mktime(timestamp.timetuple()))
start=pd.to_datetime(df['intime'].iloc[0], format='%Y-%m-%d %H:%M:%S')
start=(time.mktime(start.timetuple()))
print(start,end)

1673983477.0 1673983626.0


In [114]:
interval = (df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()
interval= round(interval)

In [115]:
interval= round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) if round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) > 60  else 60
print(interval)

149


In [116]:
import copy 
def getdataprometheus(url):
    headers= {"Accept": "application/json"}
    res = json.loads(requests.post(url=url, headers=headers).content.decode('utf8', 'ignore'))
    #data2=res.get('data').get('result')[0].get('values')
    data_dict={}
    metric_list = []
    # print(data['data']['result']['values'])
    # exit()
    for i in res['data']['result']:
        for j in i['values']:
            data_dict = copy.deepcopy(i['metric'])
            data_dict['time'] = j[0]
            data_dict['value'] = j[1]
            metric_list.append(data_dict)
    df_metric = pd.DataFrame(metric_list)
    return df_metric

In [117]:
user=df.shape[0]
metrics = {'workload_type':""+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration),'workload_size':user,'Workload TPT':(df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds(),'Average Workload TPT':df['TPT'].mean(),'Average FET':df['FET'].mean()}

promo_metrics = pd.DataFrame(columns=['cpu_used','cpu_requested'])

In [118]:

expr_namespace_cpu_requested = 'sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas-fn", resource="cpu"})&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['cpu_requested']=getdataprometheus(pre_url+expr_namespace_cpu_requested)['value']

promo_metrics['time']=getdataprometheus(pre_url+expr_namespace_cpu_requested)['time']
expr_namespace_cpu_used = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['cpu_used']=getdataprometheus(pre_url+expr_namespace_cpu_used)['value']

expr_namespace_pod_count = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn"})&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['pod_count']=getdataprometheus(pre_url+expr_namespace_pod_count)['value']

expr_namespace_memory_used = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['memory_used']=getdataprometheus(pre_url+expr_namespace_memory_used)['value']

expr_namespace_memory_requested = 'sum(rate(container_memory_working_set_bytes{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['memory_used']=getdataprometheus(pre_url+expr_namespace_memory_requested)['value']

promo_metrics['time']= pd.to_datetime(promo_metrics['time'],unit='s')
seconds = []
for i in range(promo_metrics.shape[0]): 
    seconds.append(i*10)
promo_metrics['seconds'] = seconds

throughput_qurty = 'sum (rate(gateway_function_invocation_total{code="200"}[10s]))'
promo_metrics['throughput']= getdataprometheus('http://172.17.141.197:31376/api/v1/query_range?query=sum (rate(gateway_function_invocation_total{code="200"}[20s]))&start='+str(start)+'&end='+str(end)+'&step=10s')['value']
promo_metrics.to_csv("keda_results/promo_metrics"+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv',index=False)


# #expr_namespace_cpu_requested = 'sum(node_namespace_pod_container:container_cpu_usage_seconds_total:sum_irate{namespace="openfaas-fn"})&time='+str(time_interval)
# expr_namespace_cpu_used='sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_used']=getdataprometheus(pre_url+expr_namespace_cpu_used)
# expr_namespace_cpu_requested='sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas-fn", resource="cpu"})&time='+str(time_interval)
# metrics['CPU_requested']=getdataprometheus(pre_url+expr_namespace_cpu_requested)

# expr_pod_cpu_rabbitmq_connector = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# expr_pod_memory_rabbitmq_connector = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_rabbitmq_broker_used']=getdataprometheus(pre_url+expr_pod_cpu_rabbitmq_connector)
# metrics['Memory_rabbitmq_broker_used']=getdataprometheus(pre_url+expr_pod_memory_rabbitmq_connector)

# expr_pod_cpu_rabbitmq_connector_requested = 'sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas", resource="cpu",pod=~"rabbitmq.*"})&time='+str(time_interval)
# expr_pod_memory_rabbitmq_connector = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_rabbitmq_broker_requested']=getdataprometheus(pre_url+expr_pod_cpu_rabbitmq_connector_requested)

# expr_pod_rabbitmq_function = 'sum(kube_pod_container_status_ready{namespace="openfaas",pod=~"rabbitmq.*"})&time='+str(time_interval)
# metrics['pod_count_rabbitmq']=getdataprometheus(pre_url+expr_pod_rabbitmq_function)

# expr_pod_aeneas_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"aeneas.*"})&time='+str(time_interval)
# expr_pod_tocloud_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"scaling-aeneas-tocloud.*"})&time='+str(time_interval)
# expr_pod_getobject_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"getobject.*"})&time='+str(time_interval)
# print(pre_url+expr_pod_aeneas_function)
# metrics['pod_count_aeneas']=getdataprometheus(pre_url+expr_pod_aeneas_function)
# metrics['pod_count_tocloud']=getdataprometheus(pre_url+expr_pod_tocloud_function)
# metrics['pod_count_getobject']=getdataprometheus(pre_url+expr_pod_getobject_function)

# metrics

In [119]:
df_metrics = pd.read_csv("overall_k8s.csv")
df_metrics = df_metrics.append(metrics, ignore_index=True)

In [120]:
df_metrics.to_csv("overall_k8s.csv",index=False)
display(df_metrics)

,Average Workload TPT,Workload TPT,workload_size,workload_type,Average FET
0,11.436108,141.898997,36,36_120_k8s,11.025844
1,7.101899,133.493527,36,36_120_k8s,7.058822
2,14.603441,137.332240,45,45_120_k8s,14.350427
3,5.864947,123.858352,41,41_120_k8s,5.817637
4,11.670310,146.420508,45,45_120_k8s_0.05,11.225889
5,15.487042,148.868021,42,42_120_k8s_keda_0.05,14.891617
6,10.366848,138.844865,46,46_120_k8s_keda_0.05,9.665576
7,13.272280,148.666064,44,44_120_k8s_keda_0.05,12.860280


In [121]:
# Frequency
stats_df = df \
.groupby('PT') \
['PT'] \
.agg('count') \
.pipe(pd.DataFrame) \
.rename(columns = {'PT': 'frequency'})

# PDF
stats_df['pdf'] = stats_df['frequency'] / sum(stats_df['frequency'])
stats_df['cdf'] = stats_df['pdf'].cumsum()
stats_df = stats_df.reset_index()
display(stats_df)
stats_df.to_csv("stats_df_"+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv')
# from matplotlib import pyplot as plt

# plt.plot(stats_df['PT'],stats_df['cdf'],  linestyle='--', marker='o', color='b', label='k8s_k8s')
# #plt.plot(df['user'],df['CT'],  linestyle='--', marker='*', color='r', label='Communication Time')
# plt.grid()
# plt.legend(loc='best')
# plt.xlabel('PT (Seconds)', fontsize=18)
# plt.ylabel('CDF', fontsize=16)

,PT,frequency,pdf,cdf
0,4.0,7,0.159091,0.159091
1,5.0,2,0.045455,0.204545
2,6.0,4,0.090909,0.295455
3,7.0,4,0.090909,0.386364
4,8.0,1,0.022727,0.409091
5,9.0,3,0.068182,0.477273
6,10.0,2,0.045455,0.522727
7,11.0,1,0.022727,0.545455
8,12.0,2,0.045455,0.590909
9,13.0,2,0.045455,0.636364


In [122]:
!ssh ubuntu@172.17.141.197 "sudo kubectl get hpa -n openfaas-fn"


NAME                     REFERENCE                           TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
getobject                Deployment/getobject                0%/50%    1         10        1          3m23s
aeneas                   Deployment/aeneas                   0%/30%    1         15        6          3m23s
scaling-aeneas-tocloud   Deployment/scaling-aeneas-tocloud   0%/50%    1         10        1          3m23s
pubrabbitmq              Deployment/pubrabbitmq              0%/50%    1         10        1          3m22s


In [123]:
!ssh ubuntu@172.17.141.197 "sudo kubectl delete hpa/pubrabbitmq  -n openfaas-fn;sudo kubectl delete hpa/aeneas  -n openfaas-fn;sudo kubectl delete hpa/rabbitmq-connector -n openfaas;sudo kubectl delete hpa/getobject  -n openfaas-fn;sudo kubectl delete hpa/scaling-aeneas-tocloud  -n openfaas-fn"

horizontalpodautoscaler.autoscaling "pubrabbitmq" deleted
horizontalpodautoscaler.autoscaling "aeneas" deleted
Error from server (NotFound): horizontalpodautoscalers.autoscaling "rabbitmq-connector" not found
horizontalpodautoscaler.autoscaling "getobject" deleted
horizontalpodautoscaler.autoscaling "scaling-aeneas-tocloud" deleted
